In [0]:
%sql
USE CATALOG newpavancatalog;
use schema bronze;

In [0]:
%sql
USE CATALOG newpavancatalog;
USE SCHEMA bronze;

CREATE EXTERNAL VOLUME IF NOT EXISTS volume_auto_loader_payments
    LOCATION 'abfss://landing@parkaru15sa.dfs.core.windows.net/payments';

In [0]:
%fs ls /Volumes/newpavancatalog/bronze/volume_auto_loader_payments/

In [0]:
# %python
# # Ingest raw data into Bronze
# from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType

# schema = StructType([
#     StructField("payment_id", StringType(), True),
#     StructField("order_id", StringType(), True),
#     StructField("payment_timestamp", TimestampType(), True),
#     StructField("payment_status", IntegerType(), True),
#     StructField("payment_method", StringType(), True)
# ])

# raw_df = (
#     spark.read
#     .option("delimiter", ",")
#     .option("header", "false")
#     .schema(schema)
#     .csv("abfss://landing@parkaru15sa.dfs.core.windows.net/payments/payments_2024_10.csv")
# )
# display(raw_df)

In [0]:
# %python
# raw_df.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("newpavancatalog.bronze.bronze_payments_auto_loader")

In [0]:
%sql
-- select * from newpavancatalog.bronze.bronze_payments_auto_loader

In [0]:
df = (
    spark.readStream
         .format("delta")
         .table("newpavancatalog.bronze.bronze_payments_auto_loader")
)
display(df)

In [0]:
df = spark.table("newpavancatalog.bronze.bronze_payments_auto_loader")
df.coalesce(1).write.mode("overwrite").option("header", "true").csv("/Volumes/newpavancatalog/bronze/volume_auto_loader_payments/bronze_payments_auto_loader_csv")

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, TimestampType

schema = StructType([
    StructField("payment_id", StringType(), True),
    StructField("order_id", StringType(), True),
    StructField("payment_timestamp", TimestampType(), True),
    StructField("payment_status", IntegerType(), True),
    StructField("payment_method", StringType(), True)
])

In [0]:
customers_df = (
                    spark.readStream
                         .format("cloudFiles")
                         .option("cloudFiles.format", "csv")
                         .option("header", "true")
                         .option("cloudFiles.schemaLocation", "/Volumes/newpavancatalog/bronze/volume_auto_loader_payments/_schema")
                         .option("cloudFiles.inferColumnTypes", "true")
                         .option("cloudFiles.schemaHints", "payment_timestamp TIMESTAMP")
                         .load("dbfs:/Volumes/newpavancatalog/bronze/volume_auto_loader_payments/")
)

In [0]:
display(customers_df)

In [0]:
from pyspark.sql.functions import current_timestamp, col

customers_transformed_df = (
                                customers_df.withColumn("file_path", col("_metadata.file_path"))
                                            .withColumn("ingestion_date", current_timestamp())
)